## Assignment 1

### Q1. Mutable Default Arguments
When you write `box=[]` in the function header, Python builds that list once (at the moment the `def` runs) and then keeps reusing the same list every time you call the function without passing `box`, so it quietly “remembers” past calls and the output grows each time; that’s why this pattern is infamous, and the usual fix is to use `None` as the default and create a fresh list inside the function when you actually need it.

In [1]:
def add_item_buggy(item, box=[]):
    box.append(item)
    return box

def add_item(item, box=None):
    if box is None:
        box = []
    box.append(item)
    return box

print(add_item_buggy("apple"))
print(add_item_buggy("banana"))
print(add_item_buggy("cherry"))

print(add_item("apple"))
print(add_item("banana"))
print(add_item("cherry"))

['apple']
['apple', 'banana']
['apple', 'banana', 'cherry']
['apple']
['banana']
['cherry']


### Q2. `__str__` vs `__repr__`
I treat `__repr__` as the “debug face” of an object (something precise that helps me understand what it is, and ideally something I could almost paste back into Python), while `__str__` is the “presentation face” that I’d want normal users to see; `print(obj)` uses `__str__` if you wrote one, and if you didn’t, Python falls back to `__repr__`, so defining a good `__repr__` is usually the minimum and `__str__` is the optional nicer version.

In [2]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Point(x={self.x}, y={self.y})"

    def __str__(self):
        return f"({self.x}, {self.y})"

p = Point(3, 4)
print(p)
print(repr(p))

(3, 4)
Point(x=3, y=4)


### Q3. Class Variables vs Instance Variables
A class variable is one shared value stored on the class itself (so all objects “see” it), while an instance variable is stored separately inside each object; if you do `Dog.species = ...` you change the shared value and every instance that hasn’t overridden it will reflect the update, but if you do `d1.species = ...` you’re not changing the class’s attribute at all—you’re creating a new attribute on just `d1` that shadows the class attribute, which is why `d1` can differ while `d2` still follows `Dog.species`.

In [3]:
class Dog:
    species = "Canis familiaris"
    def __init__(self, name):
        self.name = name

d1 = Dog("Fido")
d2 = Dog("Buddy")

print(d1.species, d2.species, Dog.species)

Dog.species = "Canis lupus familiaris"
print(d1.species, d2.species, Dog.species)

d1.species = "Mutant Dog"
print(d1.species, d2.species, Dog.species)

Canis familiaris Canis familiaris Canis familiaris
Canis lupus familiaris Canis lupus familiaris Canis lupus familiaris
Mutant Dog Canis lupus familiaris Canis lupus familiaris


### Q4. Parsing a Log into Online/Offline Status
The simplest way is to walk through the log left to right and keep a dictionary of the latest state for each user: split on semicolons to get events, split each event on `:` to get the user and the action, and then just set `status[user]` to `'Online'` for login and `'Offline'` for logout; if someone logs in twice or logs out twice, overwriting the value still leaves you with the correct final state because the last event wins.

In [4]:
def parse_log(log_str):
    status = {}
    for event in log_str.split(";"):
        event = event.strip()
        if not event:
            continue
        user_part, action_part = event.split(":", 1)
        user = user_part.strip()
        action = action_part.strip().lower()
        if action == "login":
            status[user] = "Online"
        elif action == "logout":
            status[user] = "Offline"
    return status

log_string = "User1: Login; User2: Login; User1: Logout; User3: Login; User2: Logout"
print(parse_log(log_string))

{'User1': 'Offline', 'User2': 'Offline', 'User3': 'Online'}


### Q5. Safe Calculator with `try/except/else/finally`
I’d structure this so everything risky (parsing numbers and dividing) sits inside `try`, then catch `ValueError` for bad numeric input and `ZeroDivisionError` for dividing by zero, print the result only in the `else` block (which runs only when nothing failed), and put the “attempt complete” line in `finally` because it should run every time whether the operation succeeded or crashed.

In [8]:
def safe_calculator():
    while True:
        try:
            first = input("Enter first number (or 'q' to quit): ")
            if first.lower() == "q":
                print("Exiting calculator.")
                break
            second = input("Enter second number: ")
            op = input("Enter operator (+, -, *, /): ")
            num1 = float(first)
            num2 = float(second)
            if op == "+":
                result = num1 + num2
            elif op == "-":
                result = num1 - num2
            elif op == "*":
                result = num1 * num2
            elif op == "/":
                result = num1 / num2
            else:
                print("Unknown operator")
                continue
        except ZeroDivisionError:
            print("Cannot divide by zero")
        except ValueError:
            print("Invalid input")
        else:
            print("Result:", result)
        finally:
            print("Execution attempt complete")

### Q6. Library System (Book + Library)
This one is really about objects talking to other objects: the library holds actual `Book` instances, and checking out or returning a book just finds the matching `Book` and flips its `is_checked_out` flag, so you can see the state change on that same object afterward; it also feels natural to raise a small custom exception when the title doesn’t exist or when someone tries to check out a book that’s already checked out.

In [5]:
class BookNotAvailableError(Exception):
    pass

class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author
        self.is_checked_out = False

    def __repr__(self):
        return f"Book({self.title!r}, {self.author!r}, {self.is_checked_out})"

class Library:
    def __init__(self):
        self.books = []

    def add_book(self, book_obj):
        self.books.append(book_obj)

    def checkout_book(self, title):
        for book in self.books:
            if book.title == title:
                if book.is_checked_out:
                    raise BookNotAvailableError(f"{title} is already checked out")
                book.is_checked_out = True
                return
        raise BookNotAvailableError(f"{title} not found")

    def return_book(self, title):
        for book in self.books:
            if book.title == title:
                book.is_checked_out = False
                return
        raise BookNotAvailableError(f"{title} not found")

lib = Library()
b1 = Book("1984", "George Orwell")
b2 = Book("The Hobbit", "J.R.R. Tolkien")
lib.add_book(b1)
lib.add_book(b2)

print(lib.books)
lib.checkout_book("1984")
print(b1.is_checked_out, b2.is_checked_out)
lib.return_book("1984")
print(b1.is_checked_out, b2.is_checked_out)

[Book('1984', 'George Orwell', False), Book('The Hobbit', 'J.R.R. Tolkien', False)]
True False
False False


### Q7. Encapsulation with Properties (Employee)
Properties let you keep a clean interface while still controlling how values behave: `email` shouldn’t be stored at all because it’s just derived from first and last name, `salary` should go through a setter so you can block negative values immediately, and having a `fullname` deleter that sets `first` and `last` to `None` is a neat “reset” behavior that still feels like normal attribute access from the outside.

In [6]:
class Employee:
    def __init__(self, first, last, salary):
        self.first = first
        self.last = last
        self._salary = salary

    @property
    def email(self):
        if self.first is None or self.last is None:
            return None
        return f"{self.first.lower()}.{self.last.lower()}@company.com"

    @property
    def fullname(self):
        if self.first is None or self.last is None:
            return None
        return f"{self.first} {self.last}"

    @fullname.setter
    def fullname(self, name):
        first, last = name.split(" ", 1)
        self.first = first
        self.last = last

    @fullname.deleter
    def fullname(self):
        self.first = None
        self.last = None

    @property
    def salary(self):
        return self._salary

    @salary.setter
    def salary(self, value):
        if value < 0:
            raise ValueError("Salary cannot be negative")
        self._salary = value

e = Employee("John", "Doe", 50000)
print(e.fullname, e.email, e.salary)
e.fullname = "Jane Smith"
e.salary = 60000
print(e.fullname, e.email, e.salary)
del e.fullname
print(e.fullname, e.email)

John Doe john.doe@company.com 50000
Jane Smith jane.smith@company.com 60000
None None


### Q8. Operator Overloading (TimeDuration)
The main trick is normalization: I convert everything into total minutes first, then use `divmod(total, 60)` so minutes always stay between 0 and 59 and hours carry the overflow; once you do that, `__add__` is just adding totals and returning a new `TimeDuration`, and `__str__` is only about formatting it nicely like `4H:05M`.

In [17]:
class TimeDuration:
    def __init__(self, hours=0, minutes=0):
        total = hours * 60 + minutes
        self.hours, self.minutes = divmod(total, 60)

    def __add__(self, other):
        if not isinstance(other, TimeDuration):
            return NotImplemented
        total = (self.hours * 60 + self.minutes) + (other.hours * 60 + other.minutes)
        return TimeDuration(0, total)

    def __str__(self):
        return f"{self.hours}H:{self.minutes:02d}M"

t1 = TimeDuration(2, 45)
t2 = TimeDuration(1, 30)
t3 = t1 + t2
t4 = TimeDuration(2, 70)
print(str(t1), str(t2), str(t3), str(t4))

2H:45M 1H:30M 4H:15M 3H:10M
